In [9]:
!pip install qiskit==0.45.0 qiskit-machine-learning==0.6.0 qiskit-aer==0.13.0 

!pip install pennylane scikit-learn imbalanced-learn pylatexenc matplotlib seaborn numpy pandas

In [10]:
import sys
print(sys.executable)


/Users/omumrania/.pyenv/versions/3.10.13/bin/python


In [11]:
import time
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from qiskit_aer import StatevectorSimulator

In [12]:
# Set global seed for reproducibility
algorithm_globals.random_seed = 42

# Load preprocessed CSVs
train_df = pd.read_csv("../output/nsl_kdd_preprocessed_train.csv").sample(n=300, random_state=42)
test_df = pd.read_csv("../output/nsl_kdd_preprocessed_test.csv").sample(n=100, random_state=42)




/var/folders/4q/4f35cqzj17574xmwsj6hclfw0000gn/T/ipykernel_71113/416067019.py:2: DeprecationWarning: The property ``qiskit.utils.algorithm_globals.QiskitAlgorithmGlobals.random_seed`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. This algorithm utility has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. You can run ``pip install qiskit_algorithms`` and import ``from qiskit_algorithms.utils`` instead. 
  algorithm_globals.random_seed = 42


In [13]:
# Extract features and labels
X_train = train_df.drop("label", axis=1).values
y_train = train_df["label"].values
X_test = test_df.drop("label", axis=1).values
y_test = test_df["label"].values



In [14]:
# Quantum Feature Map
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)

# Quantum Backend
backend = StatevectorSimulator()

# Quantum Kernel
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)



In [15]:
from sklearn.decomposition import PCA

# Reduce dimensionality to 4 or fewer features
pca = PCA(n_components=4)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(f"Reduced feature shape: {X_train.shape}")  # Should be (n_samples, 4)


Reduced feature shape: (300, 4)


In [16]:
# QSVM
print("🔧 Evaluating quantum kernel...")
start = time.time()
kernel_train = quantum_kernel.evaluate(x_vec=X_train)
kernel_test = quantum_kernel.evaluate(x_vec=X_test, y_vec=X_train)
print(f"✅ Kernel evaluation completed in {time.time() - start:.2f} seconds")



🔧 Evaluating quantum kernel...
✅ Kernel evaluation completed in 1.75 seconds


In [17]:
# Classical SVC with precomputed kernel
qsvm = SVC(kernel='precomputed')
qsvm.fit(kernel_train, y_train)



SVC(kernel='precomputed')

In [18]:
# Predictions
y_pred = qsvm.predict(kernel_test)



In [19]:
# Evaluation
print("📊 Classification Report (QSVM on NSL-KDD):")
print(classification_report(y_test, y_pred))

📊 Classification Report (QSVM on NSL-KDD):
              precision    recall  f1-score   support

           0       0.67      0.96      0.79        45
           1       0.94      0.62      0.75        55

    accuracy                           0.77       100
   macro avg       0.81      0.79      0.77       100
weighted avg       0.82      0.77      0.77       100

